# test_case_cpv_2024-08-21.ipynb


- folder for data `test_case_data` -> download separately via owncloud
- folder for configs `test_case_config`


In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import os, sys
import argparse
import datetime
import logging
from pathlib import Path
import numpy as np

sys.path.append('../')
import ppcpy
import ppcpy.io.loadConfigs as loadConfigs
import ppcpy.io.readPollyRawData as readPollyRawData
import ppcpy.interface.picassoProc as picassoProc
import ppcpy.misc.helper as helper

# import matplotlib
# import matplotlib.pyplot as plt
# import numpy as np
# import pprint

In [22]:
picasso_default_config_file = Path(
    helper.detect_path_type(Path.cwd().parent) ,'ppcpy','config','pollynet_processing_chain_config.json')

In [23]:
# for purpose of the notebook mimic the argparse interface
from types import SimpleNamespace
args = SimpleNamespace()

args.timestamp = '20240821'
args.device = 'pollyxt_cpv'
dt = datetime.datetime.strptime(args.timestamp, "%Y%m%d")

# the used config file
args.picasso_config_file = "test_case_config/pollynet_processing_chain_config_test.json"
# the data file to use
args.level0_file_to_process = f"test_case_data/{dt:%Y_%m_%d_%a}_CPV_00_00_01.nc"

In [24]:
picasso_config_dict = loadConfigs.loadPicassoConfig(args.picasso_config_file,picasso_default_config_file)
# print(picasso_config_dict['pollynet_config_link_file'])
polly_config_array = loadConfigs.readPollyNetConfigLinkTable(picasso_config_dict['pollynet_config_link_file'],timestamp=args.timestamp,device=args.device)
#print(polly_config_array)

polly_config_dict = loadConfigs.getPollyConfigfromArray(
    polly_config_array, picasso_config_dict
)

2026-02-13 15:21:12,091 - INFO - picasso_default_config_file: /mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/ppcpy/config/pollynet_processing_chain_config.json
2026-02-13 15:21:12,100 - INFO - picasso_config_file: test_case_config/pollynet_processing_chain_config_test.json
2026-02-13 15:21:12,109 - INFO - pollynet_config_link_file: test_case_config/pollynet_processing_chain_config_links.xlsx
2026-02-13 15:21:12,158 - INFO - polly_default_config_file: /mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/ppcpy/config/polly_global_config.json
2026-02-13 15:21:12,168 - INFO - polly_config_file: test_case_config/pollyxt_cpv_config_20230927.json
2026-02-13 15:21:12,175 - INFO - keys default/template file, but not in specific file {'depol_cali_mode', 'turbid_thres_par_beta_1064', 'depol_cal_time_fixed_m_start', 'minSNR_4_sigNorm', 'zLim_NR_RCS_387', 'xLim_Profi_AE', 'logbookFileName', 'turbid_thres_par_beta_532', 'ice_thres_vol_depol', 'depol_cal_time_fixed_p_start', 'overlapFile_355_total_FR', 'deltaT',

In [26]:
rawfile_fullname = args.level0_file_to_process
rawfile = helper.detect_path_type(rawfile_fullname)
rawdata_dict = readPollyRawData.readPollyRawData(rawfile)

2026-02-13 15:21:19,244 - INFO - reading nc-file: test_case_data/2024_08_21_Wed_CPV_00_00_01.nc


In [27]:
data_cube = picassoProc.PicassoProc(rawdata_dict,polly_config_dict,picasso_config_dict)

In [28]:
## reset date if date in filename differs date within nc-file 
data_cube.reset_date_infile()

## checking for correct mshots
data_cube.check_for_correct_mshots()

## setting channelTags
data_cube.setChannelTags()

## check for correct date in nc-file
data_cube.reset_date_infile()

2026-02-13 15:21:44,069 - INFO - date consistency-check... 
2026-02-13 15:21:44,073 - INFO - ... date in nc-file equals date of filename
2026-02-13 15:21:44,080 - INFO - ChannelLabels: ['FR-total-355 nm', 'FR-cross-355 nm', 'FR-387 nm', 'FR-407 nm', 'FR-total-532 nm', 'FR-cross-532 nm', 'FR-607 nm', 'FR-total-1064 nm', 'NR-total-532 nm', 'NR-607 nm', 'NR-total-355 nm', 'NR-387 nm', 'DFOV', '1058', '1064s', 'none']
2026-02-13 15:21:44,081 - WARNING - removed none tag from channel list [15]
2026-02-13 15:21:44,084 - INFO - date consistency-check... 
2026-02-13 15:21:44,086 - INFO - ... date in nc-file equals date of filename


In [29]:
## preprocessing
data_cube.preprocessing(collect_debug=False)

2026-02-13 15:21:48,659 - INFO - starting data preprocessing...
2026-02-13 15:21:48,661 - INFO - ... time conversion
2026-02-13 15:21:48,681 - WARNING - ... mShots not constant min 0 max 2999
2026-02-13 15:21:48,692 - INFO - ... Deadtime-correction (Mode: 1)


mShots_norm (2639, 15) mShots_norm [2994.59492232 2994.59492232 2994.59492232 2994.59492232 2994.59492232
 2994.59492232 2994.59492232 2994.59492232 2994.57824934 2994.57824934
 2994.57824934 2994.57824934 2994.57824934 2994.57824934 2994.57824934]


/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/preprocess/pollyPreprocess.py:125: RuntimeWarning: invalid value encountered in divide
  PCR = rawSignal * (150.0 / hRes) / mShots[:, np.newaxis, :]
2026-02-13 15:22:15,722 - INFO - ... removing background from signal
2026-02-13 15:22:26,339 - INFO - ... height bin calculations
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/preprocess/pollyPreprocess.py:644: UserWarning: no explicit representation of timezones available for np.datetime64
  data_dict['time64'] = np.array([np.datetime64(t) for t in mTime_obj])
2026-02-13 15:22:27,104 - INFO - ... mask bins with low SNR


flag 532 FR [False False False False  True False False False False False False False
 False False False]
flag 355 FR [ True False False False False False False False False False False False
 False False False]
flag 607 FR [False False False False False False  True False False False False False
 False False False]


2026-02-13 15:23:02,257 - INFO - ... mask for polarization calibration
2026-02-13 15:23:02,398 - INFO - ... calculate range-corrected Signal


(2639, 4000, 15)
ranges2d (2639, 4000)


2026-02-13 15:23:12,221 - INFO - finished data preprocessing.


In [30]:
data_cube.channel_dict

{0: 'FR-total-355 nm',
 1: 'FR-cross-355 nm',
 2: 'FR-387 nm',
 3: 'FR-407 nm',
 4: 'FR-total-532 nm',
 5: 'FR-cross-532 nm',
 6: 'FR-607 nm',
 7: 'FR-total-1064 nm',
 8: 'NR-total-532 nm',
 9: 'NR-607 nm',
 10: 'NR-total-355 nm',
 11: 'NR-387 nm',
 12: 'DFOV',
 13: '1058',
 14: '1064s'}

In [31]:
data_cube.SaturationDetect()

2026-02-13 15:23:22,617 - INFO - Saturation detection


#### First bin height correctly set?

<div class="alert alert-info">
TODO: include a plot to illustrate that
</div>

In [32]:
data_cube.polarizationCaliD90()

2026-02-13 15:23:46,530 - INFO - and even a 355 channel


starting loadGHK
data_cube keys  dict_keys(['rawfile', 'rawdata_dict', 'polly_config_dict', 'picasso_config_dict', 'device', 'location', 'date', 'num_of_channels', 'num_of_profiles', 'retrievals_highres', 'retrievals_profile', 'channel_dict', 'flags', 'flag_355_total_FR', 'flag_355_cross_FR', 'flag_355_parallel_FR', 'flag_355_total_NR', 'flag_387_total_FR', 'flag_387_total_NR', 'flag_407_total_FR', 'flag_407_total_NR', 'flag_532_total_FR', 'flag_532_cross_FR', 'flag_532_parallel_FR', 'flag_532_total_NR', 'flag_532_cross_DFOV', 'flag_532_rr_FR', 'flag_607_total_FR', 'flag_607_total_NR', 'flag_1058_total_FR', 'flag_1064_total_FR', 'flag_1064_cross_FR', 'flag_1064_total_NR', 'flagSaturation'])
Using GHK from config file
G [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
H [ 0.02041 -0.998    1.       1.      -0.01477 -0.9987   1.      -0.02439
  1.       1.       1.       1.       1.       1.      -0.996  ]
K [0.97859 1.      1.      1.      0.99343 1.      1.      0.9947  1.
 1.      1.   

/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/polarization.py:284: RuntimeWarning: divide by zero encountered in divide
  dplus = smooth_signal(sig_x_p, smooth_win) / smooth_signal(sig_t_p, smooth_win)
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/polarization.py:284: RuntimeWarning: invalid value encountered in divide
  dplus = smooth_signal(sig_x_p, smooth_win) / smooth_signal(sig_t_p, smooth_win)
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/polarization.py:285: RuntimeWarning: divide by zero encountered in divide
  dminus = smooth_signal(sig_x_m, smooth_win) / smooth_signal(sig_t_m, smooth_win)
2026-02-13 15:23:49,957 - INFO - pol_cali_355
2026-02-13 15:23:49,965 - INFO - and even a 532 channel
2026-02-13 15:23:50,308 - INFO - pol_cali_532
2026-02-13 15:23:50,312 - INFO - and even a 1064 channel


pol_cali_nang_start_time [1724207790, 1724222190]


2026-02-13 15:23:50,614 - INFO - pol_cali_1064


pol_cali_nang_start_time [1724207790, 1724222190]


In [33]:
data_cube.cloudScreen()

Starting cloud screen
cloud screen mode 1
slope_thres 70000.0


In [34]:
data_cube.cloudFreeSeg()

intNProfiles 120 minIntNProfiles 30


In [35]:
data_cube.clFreeGrps

array([[  0, 113],
       [144, 247],
       [340, 369]])

In [36]:
# data_cube.clFreeGrps = [
#     [35, 300],
#     [1000, 1300],
#     [2650, 2870]
# ]

In [37]:
data_cube.aggregate_profiles()

In [38]:
data_cube.loadMeteo()

In [39]:
data_cube.calcMolecular()

time slices of cloud free  [array(['2024-08-21T00:00:00.000000', '2024-08-21T00:56:30.000000'],
      dtype='datetime64[us]'), array(['2024-08-21T01:12:00.000000', '2024-08-21T02:03:30.000000'],
      dtype='datetime64[us]'), array(['2024-08-21T02:50:00.000000', '2024-08-21T03:04:30.000000'],
      dtype='datetime64[us]')]
len mean_profiles 3
shape of the molecular scattering (3, 4000)
for the wavelengths  [355, 387, 407, 532, 607, 1058, 1064]


In [40]:
data_cube.rayleighFit()

2026-02-13 15:24:27,878 - WARNING - Potential for differences to matlab code du to numerical issues (subtraction of two small values)
2026-02-13 15:24:27,881 - WARNING - rayleighfit seems to use range in matlab, but the met data should be in height >> RECHECK!
2026-02-13 15:24:27,883 - WARNING - at 10km height this is a difference of about 4 indices


Start Rayleigh Fit
0 [  0 113]
refH for 532


/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/rayleighfit.py:403: RuntimeWarning: divide by zero encountered in divide
  std_aer_norm = sig_aer_norm / np.sqrt(pc + bg)


refHInd (1734, 2135)
refH for 355
refHInd (1765, 2368)
refH for 1064
one tests failed?
refHInd (1872, 1926)
1 [144 247]
refH for 532
refHInd (1332, 1733)
refH for 355
refHInd (1765, 2368)
refH for 1064
one tests failed?
one tests failed?
refHInd (nan, nan)
2 [340 369]
refH for 532
refHInd (1734, 2135)
refH for 355
refHInd (962, 1764)
refH for 1064
refHInd (nan, nan)


[{'532_total_FR': {'DPInd': array([ 126,  527,  528,  929,  930, 1331, 1332, 1733, 1734, 2135, 2136,
          2388]),
   'refHInd': (1734, 2135)},
  '355_total_FR': {'DPInd': array([ 159,  961,  962, 1764, 1765, 2368]),
   'refHInd': (1765, 2368)},
  '1064_total_FR': {'DPInd': array([ 121,  388,  389,  656,  657,  689,  956,  957, 1224, 1225, 1492,
          1493, 1760, 1761, 1818, 1838, 1845, 1872, 1926, 1958, 1960, 2005,
          2059, 2068, 2074, 2085, 2098, 2164, 2206, 2320, 2354, 2396, 2463,
          2479, 2527, 2549, 2586, 2606, 2610, 2612, 2623, 2685, 2688, 2798,
          2881, 2886, 2915, 2930, 2937, 2955, 2958, 2980, 3000, 3004, 3032,
          3047, 3049, 3062, 3077, 3090, 3091, 3093, 3112, 3159, 3216, 3297,
          3325]),
   'refHInd': (1872, 1926)}},
 {'532_total_FR': {'DPInd': array([ 126,  527,  528,  929,  930, 1331, 1332, 1733, 1734, 2135, 2136,
          2388]),
   'refHInd': (1332, 1733)},
  '355_total_FR': {'DPInd': array([ 159,  961,  962, 1764, 1765, 2368]),

In [41]:
# Use config values for refH in NR channels (similar approche to Picasso)
for e in data_cube.refH:
    e['532_total_NR'] = {'DPInd':None, 'refHInd':tuple(np.searchsorted(data_cube.retrievals_highres["height"], data_cube.polly_config_dict["refH_NR_532"]) - [0, 1])}     # - [0, 1] --> make sure all values lay inside the designeted range
    e['355_total_NR'] = {'DPInd':None, 'refHInd':tuple(np.searchsorted(data_cube.retrievals_highres["height"], data_cube.polly_config_dict["refH_NR_355"]) - [0, 1])}     # - [0, 1] --> make sure all values lay inside the designeted range

In [42]:
data_cube.transCor()

2026-02-13 15:24:36,388 - WARNING - transmission correction
2026-02-13 15:24:38,127 - INFO - and even a 355 channel


G [1.] [1.]
H [0.02041] [-0.998]
polCaliEta 46.770013651088924
G [1.] [1.] H [0.02041] [-0.998] Eta 46.770013651088924 error [ 0.00016  0.01567 -0.00958] Window 1 


/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/depolarization.py:161: RuntimeWarning: divide by zero encountered in divide
  sig_ratio = sigc / sigt
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/depolarization.py:161: RuntimeWarning: invalid value encountered in divide
  sig_ratio = sigc / sigt
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/depolarization.py:171: RuntimeWarning: invalid value encountered in divide
  vol_depol = (sig_ratio / eta * (Gt + Ht) - (Gr + Hr)) / ((Gr - Hr) - sig_ratio / eta * (Gt - Ht))


calculated R_t [0.95999647]


2026-02-13 15:24:43,606 - INFO - and even a 532 channel


G [1.] [1.]
H [-0.01477] [-0.9987]
polCaliEta 12.397702207784498
G [1.] [1.] H [-0.01477] [-0.9987] Eta 12.397702207784498 error [ 0.00027  0.02024 -0.00463] Window 1 


2026-02-13 15:24:46,371 - INFO - and even a 1064 channel


calculated R_t [1.02998285]
G [1.] [1.]
H [-0.02439] [-0.996]
polCaliEta 0.14655543265388163
G [1.] [1.] H [-0.02439] [-0.996] Eta 0.14655543265388163 error [ 0.00133  0.02379 -0.01205] Window 1 
calculated R_t [1.04999949]


In [43]:
data_cube.aggregate_profiles(var='sigTCor')
data_cube.aggregate_profiles(var='BGTCor')

In [44]:
data_cube.retrievalKlett(nr=True)

2026-02-13 15:24:54,476 - WARNING - rayleighfit seems to use range in matlab, but the met data should be in height >> RECHECK!
2026-02-13 15:24:54,479 - WARNING - at 10km height this is a difference of about 4 indices


retrievalname klett
Starting Klett retrieval
cldFree  0 [  0 113]
cldFree mod (np.int64(0), np.int64(114))
== 532, total, FR klett =================================
refHInd (1734, 2135) refH [12965.39983463 15962.87479639]
LR  39 refH [12965.39983463 15962.87479639] refBeta 1e-08 smoothWin_klett 25
== 355, total, FR klett =================================
refHInd (1765, 2368) refH [13197.12483168 17704.54977417]
LR  48 refH [13197.12483168 17704.54977417] refBeta 5e-08 smoothWin_klett 25
== 1064, total, FR klett =================================
refHInd (1872, 1926) refH [13996.94982147 14400.59981632]
LR  30 refH [13996.94982147 14400.59981632] refBeta 5e-09 smoothWin_klett 25
== 532, total, NR klett =================================
refHInd (np.int64(403), np.int64(536)) refH [3016.17496157 4010.34994888]
LR  25 refH [3016.17496157 4010.34994888] refBeta 1.392317191945856e-06 smoothWin_klett 13
== 355, total, NR klett =================================
refHInd (np.int64(403), np.int64

/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/klettfernald.py:310: RuntimeWarning: invalid value encountered in divide
  aerRelBRStd = np.abs((1 + noise / signal) / (1 + aerBsc + molBsc / 1e3) - 1)


In [45]:
data_cube.retrievalRaman(nr=True)

2026-02-13 15:24:59,115 - WARNING - rayleighfit seems to use range in matlab, but the met data should be in height >> RECHECK!
2026-02-13 15:24:59,117 - WARNING - at 10km height this is a difference of about 4 indices


[array([900, 900, 900, 900, 800, 800, 800, 800, 150, 150, 150, 150, 150,
       800, 800]), array([900, 900, 900, 900, 800, 800, 800, 800, 150, 150, 150, 150, 150,
       800, 800]), array([900, 900, 900, 900, 800, 800, 800, 800, 150, 150, 150, 150, 150,
       800, 800])]
Starting Raman retrieval
cldFree  0 [  0 113]
cldFree mod (np.int64(0), np.int64(114))
== 355, total, FR | 387, total, FR raman ========


/mnt/c/Users/radenz/dev/PicassoPy/pyppc_env/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


900 370.01248
refHInd (1765, 2368) refH [13197.12483168 17704.54977417] hBaseInd 170 hBase 1274.4999837875366
filling aerExt below overlap with 0.00012114657363209074 for calculating the backscatter


/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/raman.py:494: RuntimeWarning: invalid value encountered in sqrt
  sigElasticSample = sigGenWithNoise(sigElastic, np.sqrt(sigElastic + bgElastic), MC_count[2], 'norm').T


== 532, total, FR | 607, total, FR raman ========
800 370.01248
refHInd (1734, 2135) refH [12965.39983463 15962.87479639] hBaseInd 157 hBase 1177.3249850273132
filling aerExt below overlap with 0.00011563465900385698 for calculating the backscatter
== 1064, total, FR | 607, total, FR raman ========
800 370.01248
refHInd (1872, 1926) refH [13996.94982147 14400.59981632] hBaseInd 157 hBase 1177.3249850273132
filling aerExt below overlap with 9.392452887771897e-05 for calculating the backscatter
== 532, total, NR | 607, total, NR raman ========
150 190.6125
refHInd (np.int64(403), np.int64(536)) refH [3016.17496157 4010.34994888] hBaseInd 46 hBase 347.59999561309814
filling aerExt below overlap with 0.00018415933213712955 for calculating the backscatter


/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/raman.py:345: RuntimeWarning: invalid value encountered in sqrt
  noise = np.sqrt(sig + bg)
/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/retrievals/raman.py:495: RuntimeWarning: invalid value encountered in sqrt
  sigVRN2Sample = sigGenWithNoise(sigVRN2, np.sqrt(sigVRN2 + bgVRN2), MC_count[2], 'norm').T


== 355, total, NR | 387, total, NR raman ========
150 190.6125
refHInd (np.int64(403), np.int64(536)) refH [3016.17496157 4010.34994888] hBaseInd 46 hBase 347.59999561309814
filling aerExt below overlap with 0.00018354335672136175 for calculating the backscatter
cldFree  1 [144 247]
cldFree mod (np.int64(144), np.int64(248))
== 355, total, FR | 387, total, FR raman ========
900 370.01248
refHInd (1765, 2368) refH [13197.12483168 17704.54977417] hBaseInd 170 hBase 1274.4999837875366
filling aerExt below overlap with 0.00010321594124420529 for calculating the backscatter
== 532, total, FR | 607, total, FR raman ========
800 370.01248
refHInd (1332, 1733) refH [ 9960.44987297 12957.92483473] hBaseInd 157 hBase 1177.3249850273132
filling aerExt below overlap with 9.964771803244279e-05 for calculating the backscatter
== 1064, total, FR | 607, total, FR raman ========
No valid refHInd found, skipping Raman retrieval for this channel.
== 532, total, NR | 607, total, NR raman ========
150 190.

### skipped the rest of the optical profiles and the OL correction for now

In [46]:
data_cube.LidarCalibration()

retival klett
LCMeanWindow 50 LCMeanMinIndx 149 LCMeanMaxIndx 999
cldFreGrp 0, Channel 532 total FR, LC_stable 112999123162583.52, LCStd 0.0022334435096273237
cldFreGrp 0, Channel 355 total FR, LC_stable 26551468228461.727, LCStd 0.001698456995108788
cldFreGrp 0, Channel 1064 total FR, LC_stable 76368995532193.69, LCStd 0.002984430788167574
cldFreGrp 0, Channel 532 total NR, LC_stable 7483086984445.365, LCStd 0.0014177362837897584
cldFreGrp 0, Channel 355 total NR, LC_stable 2637959386608.313, LCStd 0.0017817304655695677
cldFreGrp 1, Channel 532 total FR, LC_stable 104218815314663.22, LCStd 0.001945203426111636
cldFreGrp 1, Channel 355 total FR, LC_stable 23527881516226.42, LCStd 0.0025568686641991613
cldFreGrp 1, Channel 532 total NR, LC_stable 7607004523069.57, LCStd 0.002143120090185318
cldFreGrp 1, Channel 355 total NR, LC_stable 2633518651323.879, LCStd 0.0015614146983692603
cldFreGrp 2, Channel 532 total FR, LC_stable 115622952577898.06, LCStd 0.0026479525714390227
cldFreGrp 2, C

/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/misc/helper.py:758: RuntimeWarning: Mean of empty slice
  thisMean = np.nanmean(window)


cldFreGrp 0, Channel 355 total NR, LC_stable 2995475967842.4575, LCStd 0.004162779382638682
cldFreGrp 0, Channel 387 total NR, LC_stable 3069414415261.6133, LCStd 0.001475493357720307
cldFreGrp 1, Channel 355 total FR, LC_stable 18046775759246.03, LCStd 0.0033613778783274494
cldFreGrp 1, Channel 387 total FR, LC_stable 36329028558257.56, LCStd 0.0006533511949093894
cldFreGrp 1, Channel 532 total FR, LC_stable 97302881236048.84, LCStd 0.005767131389876474
cldFreGrp 1, Channel 607 total FR, LC_stable 180778571239397.56, LCStd 0.0007882941767295437
skipping 1064_total_FR (np.int64(144), np.int64(248))
cldFreGrp 1, Channel 532 total NR, LC_stable 9816969076372.69, LCStd 0.005076879846308099
cldFreGrp 1, Channel 607 total NR, LC_stable 8666713796504.171, LCStd 0.002285543244421737


/mnt/c/Users/radenz/dev/PicassoPy/PicassoPy/tests/../ppcpy/calibration/lidarconstant.py:78: RuntimeWarning: invalid value encountered in divide
  LC = signal * height**2 / bsc / trans


cldFreGrp 1, Channel 355 total NR, LC_stable 3015988948020.9824, LCStd 0.004070002352127664
cldFreGrp 1, Channel 387 total NR, LC_stable 3011974252152.176, LCStd 0.0016480729850006067
cldFreGrp 2, Channel 355 total FR, LC_stable 19849454063286.95, LCStd 0.006288194042260955
cldFreGrp 2, Channel 387 total FR, LC_stable 39179401872816.79, LCStd 0.0014254153249812131
cldFreGrp 2, Channel 532 total FR, LC_stable 108888849328792.42, LCStd 0.0049055827969260585
cldFreGrp 2, Channel 607 total FR, LC_stable 188541406347273.62, LCStd 0.001125496479043721
skipping 1064_total_FR (np.int64(340), np.int64(370))
cldFreGrp 2, Channel 532 total NR, LC_stable 10403142320805.203, LCStd 0.014126614449606093
cldFreGrp 2, Channel 607 total NR, LC_stable 8782120154012.966, LCStd 0.004258105012044573
cldFreGrp 2, Channel 355 total NR, LC_stable 3179059800093.6675, LCStd 0.005129040397766813


2026-02-13 15:25:22,157 - WARNING - reading calibration constant from database not working yet


cldFreGrp 2, Channel 387 total NR, LC_stable 3075571239011.529, LCStd 0.004429286210877878


In [48]:
data_cube.LCused

{'532_total_FR': np.float64(118300666337131.23),
 '355_total_NR': np.float64(3015988948020.9824),
 '355_total_FR': np.float64(18046775759246.03),
 '1064_total_FR': np.float64(80216970984202.34),
 '532_total_NR': np.float64(9816969076372.69),
 '387_total_NR': np.float64(3069414415261.6133),
 '607_total_FR': np.float64(180778571239397.56),
 '607_total_NR': np.float64(8666713796504.171),
 '387_total_FR': np.float64(36329028558257.56)}

#### Result of `data_cube.LCused`

```
{'532_total_FR': np.float64(118300666337131.23),
 '355_total_NR': np.float64(3015988948020.9824),
 '355_total_FR': np.float64(18046775759246.03),
 '1064_total_FR': np.float64(80216970984202.34),
 '532_total_NR': np.float64(9816969076372.69),
 '387_total_NR': np.float64(3069414415261.6133),
 '607_total_FR': np.float64(180778571239397.56),
 '607_total_NR': np.float64(8666713796504.171),
 '387_total_FR': np.float64(36329028558257.56)}
```